In [1]:
import tensorflow as tf
tf.enable_eager_execution()

In [21]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# vocab set and vocab size
vocab = set(raw_text)
vocab_size = len(vocab)

# construct dictionary to lookup 
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {ix: word for word, ix in word_to_ix.items()}

raw_skipgram_data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    center = raw_text[i]
    for word in context: 
        raw_skipgram_data.append((center,word))
print(raw_skipgram_data[:5])


[('about', 'We'), ('about', 'are'), ('about', 'to'), ('about', 'study'), ('to', 'are')]


In [75]:
target =  [word_to_ix[data[1]] for data in raw_skipgram_data]
data = tf.constant([word_to_ix[data[0]] for data in raw_skipgram_data])
label = tf.one_hot(target, depth=vocab_size)

In [76]:
print(data[0])
print(label[0])

tf.Tensor(43, shape=(), dtype=int32)
tf.Tensor(
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0.], shape=(49,), dtype=float32)


In [126]:
class CBOW(tf.keras.models.Model):
    def __init__(self):
        super(CBOW,self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, 5)
        self.dense = tf.keras.layers.Dense(vocab_size, activation='softmax', use_bias=False)
    
    def call(self, input):
        x = self.embedding(input)[:,0,:]
        x = self.dense(x)
        return x

In [127]:
model = CBOW()

In [128]:
criterion = tf.keras.losses.CategoricalCrossentropy()

In [129]:
optimizer = tf.keras.optimizers.SGD(0.01, momentum=0.01)

In [130]:
model.compile(optimizer=optimizer, loss=criterion, metrics=["accuracy"])

In [131]:
model.build(input_shape=(1,))

ValueError: Index out of range using input dim 2; input has only 2 dims for 'strided_slice' (op: 'StridedSlice') with input shapes: [1,5], [3], [3], [3] and with computed input tensors: input[3] = <1 1 1>.